In [1]:
import os
import sys 
from flask import Flask
from flask import request
from flask_api import status
import psycopg2
import pybars
import json
import copy
from manejo_base_de_datos import baseDatos
from manejo_archivos_json import dataJson
from IPython.core.display import display, HTML
manejadorBaseDeDatos=baseDatos("postgres","carlos98","127.0.0.1","5432","chatblocks")#manejador de consultas a base de datos 
manejadorArchivos = dataJson(os.getcwd()+"/") 
display(HTML("<style>.container { width:100% !important; }</style>"))
datosBaseDeDatos = {"usuario":"postgres","password":"carlos98","Host":"127.0.0.1","puerto":"5432","DB":"chatblocks"}

In [2]:
def compiladorPybars(source,jsonData):
    compiler = pybars.Compiler()
    template = compiler.compile(source)
    output = template(jsonData)
    json_acceptable_string = output.replace("'", "\"")
    datos = json.loads(json_acceptable_string)
    return datos

In [3]:
def cargarVariable(datosVariable,id_robot):
    datosVariable.update({"id_robot":id_robot})
    variable = manejadorBaseDeDatos.insertDatos(datosVariable["variable"],"variables")
    return int(variable["id_var"])
    

In [4]:
def obtenerDatosQR(datosJson):    
    contenido=datosJson["content"]
    opciones=list()
    outputs=list()
    for i in range(len(datosJson["buttons"])):
        opciones.append(datosJson["buttons"][i]["title"])
        outputs.append(datosJson["buttons"][i]["next_block"])
    datos={"contenido":contenido,
           "opciones":",".join(opciones),
           "next_id":",".join(outputs)        
         }
    return datos

In [5]:
def obtenerDatosSlide(bloque):    
    datosBloqueSlide = {"namestate":bloque["name"],"blocktype":bloque["type"],"typingtime":bloque["parameters"]["typingtime"],"default_id":bloque["outputs"]["default_id"],"id_var":bloque["parameters"]["id_var"],"id_robot":bloque["parameters"]["id_robot"]}
    bloqueSlide  = manejadorBaseDeDatos.insertDatos(datosBloqueSlide,"bloqueslide")
    idBloqueSlide = bloqueSlide["id_block"]    
    elementos = copy.deepcopy(bloque["parameters"]["json"])
    next_id=list()
    print("datosElemento")
    for elemento in range(len(elementos)):
        datosElemento = copy.deepcopy(elementos[elemento])
        datosElemento.update({"blocktype":bloque["type"]})
        datosElemento.update({"id_block":idBloqueSlide})
        del datosElemento["buttons"]   
        datosElementoBase =  manejadorBaseDeDatos.insertDatos(datosElemento,"elementos")
        idElemento=datosElementoBase["id_elements"]                
        botones=copy.deepcopy(elementos[elemento]["buttons"])
        for b in range(len(botones)):
            datosBoton = copy.deepcopy(botones[b])
            datosBoton.update({"id_elemento":str(idElemento)})
            datosBoton.update({"contentbutton":botones[b]["opc_nextid"]})     
            datosBoton={"id_elemento":str(idElemento),"titlebutton":botones[b]["titlebutton"],"typebutton":botones[b]["typebutton"],"contentbutton":botones[b]["opc_nextid"],"opc_nextid":botones[b]["opc_nextid"]}
            boton = manejadorBaseDeDatos.insertDatos(datosBoton,"botones") 
            next_id.append(botones[b]["opc_nextid"])
    next_id=",".join(next_id)
    tabla = "bloqueslide"
    datosActualizar={"next_id":next_id}
    datos={"id_block":idBloqueSlide}            
    tabla = "bloqueslide"   
    manejadorBaseDeDatos.update(datosActualizar,datos,tabla)   

In [6]:
def eraseChatbot ( botData ):
    respuesta = manejadorBaseDeDatos.getDatos(botData,"robots")
    print("Hasta aqui todo bien")
    hayRobot = True
    if "error" not in respuesta:
        datosRobot = {"id_robot", respuesta["id_robot"]}
    else:
        hayRobot = False
    if not hayRobot:
        datosBot= manejadorBaseDeDatos.insertDatos(botData,"robots")
        idBot=int(datosBot["id_robot"])
    else:
        idBot=respuesta["id_robot"]
        datosBot = {"id_robot": idBot}
    for bloque,datosBloque in bloquesBot.items():
        datosBloque["parameters"].update({"id_robot":idBot})   
    if hayRobot:
        print(datosRobot)
        resp = manejadorBaseDeDatos.getDatos({"id_robot": str(respuesta["id_robot"])}, "bloqueslide")
        if resp.get("id_block"):
            id_block = resp["id_block"]
            resp = manejadorBaseDeDatos.getDatos({"id_block": id_block}, "elementos")
            print("ID_BLOCK" + str(id_block))
            if resp.get("id_elements"):
                id_elemento = resp["id_elements"];
                print("ID_ELEMENTO" + str(id_elemento))
                manejadorBaseDeDatos.borrarBotonesElementos( str(id_elemento), str(id_block) )
        manejadorBaseDeDatos.borrarBaseDatosPorRobot(str(datosBot["id_robot"]))
        return {"message": "Se elimino el chatbot"}
        

In [7]:
def addChatbot( botData, bloquesBot):
    respuesta = manejadorBaseDeDatos.getDatos(botData,"robots")
    
    hayRobot = True
    if "error" not in respuesta:
        datosRobot = {"id_robot", respuesta["id_robot"]}
    else:
        hayRobot = False
    if not hayRobot:
        datosBot= manejadorBaseDeDatos.insertDatos(botData,"robots")
        idBot=int(datosBot["id_robot"])
    else:
        idBot=respuesta["id_robot"]
        datosBot = {"id_robot": idBot}
    for bloque,datosBloque in bloquesBot.items():
        datosBloque["parameters"].update({"id_robot":idBot})   
    if hayRobot:
        print(datosRobot)
        print("todo bien hasta aqui")
        respAux = manejadorBaseDeDatos.getDatos({"id_robot": str(respuesta["id_robot"])}, "bloqueslide")
        print("resp")
        print(respAux)
        if isinstance( respAux, list ):
            for resp in respAux:
                if resp.get("id_block"):
                    id_block = resp["id_block"]
                    resp = manejadorBaseDeDatos.getDatos({"id_block": id_block}, "elementos")
                    if isinstance(resp,list):
                        for r in resp:
                            print("R")
                            print(r)
                            print("ID_BLOCK" + str(id_block))
                            if r.get("id_elements"):
                                id_elemento = r["id_elements"];
                                print("ID_ELEMENTO" + str(id_elemento))
                                manejadorBaseDeDatos.borrarBotonesElementos( str(id_elemento), str(id_block) )
                    else:
                        print("ID_BLOCK" + str(id_block))
                        if resp.get("id_elements"):
                            id_elemento = resp["id_elements"];
                            print("ID_ELEMENTO" + str(id_elemento))
                            manejadorBaseDeDatos.borrarBotonesElementos( str(id_elemento), str(id_block) )
        else:
            resp = respAux
            if resp.get("id_block"):
                    id_block = resp["id_block"]
                    resp = manejadorBaseDeDatos.getDatos({"id_block": id_block}, "elementos")
                    if isinstance(resp,list):
                        for r in resp:
                            print("R")
                            print(r)
                            print("ID_BLOCK" + str(id_block))
                            if r.get("id_elements"):
                                id_elemento = r["id_elements"];
                                print("ID_ELEMENTO" + str(id_elemento))
                                manejadorBaseDeDatos.borrarBotonesElementos( str(id_elemento), str(id_block) )
                    else:
                        print("ID_BLOCK" + str(id_block))
                        if resp.get("id_elements"):
                            id_elemento = resp["id_elements"];
                            print("ID_ELEMENTO" + str(id_elemento))
                            manejadorBaseDeDatos.borrarBotonesElementos( str(id_elemento), str(id_block) )
        print("todo bien hasta aqui 2")
        manejadorBaseDeDatos.borrarBaseDatosPorRobot(str(datosBot["id_robot"]))
        print(datosBot["id_robot"])
        print("todo bien hasta aqui 3")
        
    if hayRobot:
        print("si hay robot")
        print(bloquesBot.keys())
        for bot in bloquesBot.keys():    
            bloque = dict() 
            if bloquesBot[bot]["parameters"].get("jsonVariable"):  
                print(bloquesBot[bot]["parameters"]["jsonVariable"]["variable"])
                bloquesBot[bot]["parameters"]["jsonVariable"]["variable"].update({"id_robot":bloquesBot[bot]["parameters"]["id_robot"]})    
                print(type(bloquesBot[bot]["parameters"]["jsonVariable"]["variable"]))
                variable = manejadorBaseDeDatos.insertDatos(bloquesBot[bot]["parameters"]["jsonVariable"]["variable"],"variables")         
                bloquesBot[bot]["parameters"].update({"id_var":int(variable["id_var"])})  
                del bloquesBot[bot]["parameters"]["jsonVariable"]
            if bloquesBot[bot]["parameters"].get("nameVariable"):
                datos=dict()
                datos.update({"id_robot":bloquesBot[bot]["parameters"].get("id_robot")})
                datos.update({"name_var":bloquesBot[bot]["parameters"].get("nameVariable")})
                print(datos)
                print(type(datos))
                variable = manejadorBaseDeDatos.getDatos(datos,"variables")
                bloquesBot[bot]["parameters"].update({"id_var":int(variable["id_var"])})
                del bloquesBot[bot]["parameters"]["nameVariable"] 

            if bloquesBot[bot]["type"] == "quickReply": 
                del bloquesBot[bot]["parameters"]["content"]
                del bloquesBot[bot]["parameters"]["buttonTrue"]
                del bloquesBot[bot]["parameters"]["buttonFalse"]
                del bloquesBot[bot]["parameters"]["typeVisual"]
                del bloquesBot[bot]["parameters"]["handlebars"]       
                QR = obtenerDatosQR(bloquesBot[bot]["parameters"]["json"])  
                bloque.update(QR)
                
                del bloquesBot[bot]["parameters"]["json"]       

            elif bloquesBot[bot]["type"] == "slide":
                del bloquesBot[bot]["parameters"]["content"]
                #del bloquesBot[bot]["parameters"]["buttonTrue"]
                #del bloquesBot[bot]["parameters"]["buttonFalse"]
                #del bloquesBot[bot]["parameters"]["typeVisual"]
                #del bloquesBot[bot]["parameters"]["handlebars"]
                
                slide= obtenerDatosSlide(bloquesBot[bot])
                
                del bloquesBot[bot]["parameters"]["json"]  
            elif bloquesBot[bot]["type"] == "nlp":
                dataAux = copy.deepcopy( bloquesBot[bot]["parameters"]["json_nlp"])
                bloquesBot[bot]["parameters"]["json_nlp"] = str(json.dumps(dataAux))
                print("\n")
                print( bloquesBot[bot]["parameters"]["json_nlp"] )
            for salida in bloquesBot[bot]["outputs"]:
                if type(bloquesBot[bot]["outputs"][salida]) != list:
                    bloque.update({salida:bloquesBot[bot]["outputs"][salida]})
            bloque.update({"namestate":bloquesBot[bot]["name"]})
            bloque.update(bloquesBot[bot]["parameters"])
            bloque.update({"blocktype":bloquesBot[bot]["type"]})    
            if bloquesBot[bot]["type"] == "slide":
                pass
            else:
                print("Bloque:")
                print(bloque)
                print("Bloques Bot:")
                print(bloquesBot)
                print("BOT")
                print(bot)
                manejadorBaseDeDatos.insertDatos(bloque,"bloque"+bloquesBot[bot]["type"].lower())
        return {"message": "Se actualizo el chatbot"}
    else:      
        print("no hay robot")
        print(bloquesBot.keys())
        for bot in bloquesBot.keys():    
            bloque = dict() 
            print("todo bien 3")
            if bloquesBot[bot]["parameters"].get("jsonVariable"):  
                print(bloquesBot[bot]["parameters"]["jsonVariable"]["variable"])
                bloquesBot[bot]["parameters"]["jsonVariable"]["variable"].update({"id_robot":bloquesBot[bot]["parameters"]["id_robot"]})    
                print(type(bloquesBot[bot]["parameters"]["jsonVariable"]["variable"]))
                variable = manejadorBaseDeDatos.insertDatos(bloquesBot[bot]["parameters"]["jsonVariable"]["variable"],"variables")         
                bloquesBot[bot]["parameters"].update({"id_var":int(variable["id_var"])})  
                del bloquesBot[bot]["parameters"]["jsonVariable"]
            if bloquesBot[bot]["parameters"].get("nameVariable"):
                datos=dict()
                print("todo bien 2")
                datos.update({"id_robot":bloquesBot[bot]["parameters"].get("id_robot")})
                datos.update({"name_var":bloquesBot[bot]["parameters"].get("nameVariable")})
                variable = manejadorBaseDeDatos.getDatos(datos,"variables")
                bloquesBot[bot]["parameters"].update({"id_var":int(variable["id_var"])})
                del bloquesBot[bot]["parameters"]["nameVariable"] 

            if bloquesBot[bot]["type"] == "quickReply": 
                del bloquesBot[bot]["parameters"]["content"]
                del bloquesBot[bot]["parameters"]["buttonTrue"]
                del bloquesBot[bot]["parameters"]["buttonFalse"]
                del bloquesBot[bot]["parameters"]["typeVisual"]
                del bloquesBot[bot]["parameters"]["handlebars"]       
                QR = obtenerDatosQR(bloquesBot[bot]["parameters"]["json"])  
                bloque.update(QR)
                del bloquesBot[bot]["parameters"]["json"]       

            elif bloquesBot[bot]["type"] == "slide":
                print("todo bien")
                del bloquesBot[bot]["parameters"]["content"]
                #del bloquesBot[bot]["parameters"]["buttonTrue"]
                #del bloquesBot[bot]["parameters"]["buttonFalse"]
                #del bloquesBot[bot]["parameters"]["typeVisual"]
                #del bloquesBot[bot]["parameters"]["handlebars"]
                
                slide= obtenerDatosSlide(bloquesBot[bot])
                
                del bloquesBot[bot]["parameters"]["json"]  
            elif bloquesBot[bot]["type"] == "nlp":
                dataAux = copy.deepcopy( bloquesBot[bot]["parameters"]["json_nlp"])
                bloquesBot[bot]["parameters"]["json_nlp"] = str(json.dumps(dataAux))
                print("\n")
                print( bloquesBot[bot]["parameters"]["json_nlp"] )
            for salida in bloquesBot[bot]["outputs"]:
                if type(bloquesBot[bot]["outputs"][salida]) != list:
                    bloque.update({salida:bloquesBot[bot]["outputs"][salida]})
            bloque.update({"namestate":bloquesBot[bot]["name"]})
            bloque.update(bloquesBot[bot]["parameters"])
            bloque.update({"blocktype":bloquesBot[bot]["type"]})    
            if bloquesBot[bot]["type"] == "slide":
                pass
            else:        
                manejadorBaseDeDatos.insertDatos(bloque,"bloque"+bloquesBot[bot]["type"].lower())
        return {"message": "Se creo el chatbot"}
    print("bot cargado con exito ")
        

In [8]:
app = Flask(__name__)
@app.route('/insert',methods=['POST'])
def funcionalidad():
    try:
        datos = request.get_json()
        print(datos)
        botData = datos["chatbotFB"]
        bloquesBot = datos["bloques"]
        print("Datos Recibidos chatbot:")
        print(botData)
        print("Datos Recibidos bloques:")
        print(bloquesBot)
        respuesta = addChatbot( botData, bloquesBot)
        print(respuesta["message"])
        print( "Datos del ChatBot:")
        print(botData)
        return respuesta
    except (Exception, psycopg2.Error) as error :
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(exc_type, fname, exc_tb.tb_lineno)
        return 'bad request!', 400

In [ ]:
if __name__ == "__main__":    
    app.run(host='localhost',port='9595')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:9595/ (Press CTRL+C to quit)


{'bloques': {'Carrusel | 5aaf3158-308e-4b01-9283-f01f32d587c9 | 96628859-0899-7537-4503-489244348545 | ZG7udwy8rzhE5xtUnOGl': {'type': 'slide', 'name': 'Carrusel | 5aaf3158-308e-4b01-9283-f01f32d587c9 | 96628859-0899-7537-4503-489244348545 | ZG7udwy8rzhE5xtUnOGl', 'parameters': {'typingtime': '1', 'content': '', 'nombreVariable': 'variableCarrCarrusel_|_5aaf3158-308e-4b01-9283-f01f32d587c9', 'jsonVariable': {'variable': {'name_var': 'variableCarrCarrusel_|_5aaf3158-308e-4b01-9283-f01f32d587c9', 'opc_type': 'Variable', 'var': 0, 'opc_data': 'String'}}, 'tipoVariable': 'Variable', 'tipoDato': 'Cadena', 'id_var': 'variableCarrCarrusel_|_5aaf3158-308e-4b01-9283-f01f32d587c9', 'json': [{'buttons': [{'titlebutton': 'Opcion 1', 'typebutton': 'postback', 'opc_nextid': 'QuickReply | 9cd75eb1-5f87-4a23-84aa-d6de05ed56b8 | 96628859-0899-7537-4503-489244348545 | ZG7udwy8rzhE5xtUnOGl'}, {'titlebutton': 'Opcion 2', 'typebutton': 'postback', 'opc_nextid': 'QuickReply | f962f2fb-1047-4062-bac8-798fddc

127.0.0.1 - - [14/Jun/2021 23:45:35] "POST /insert HTTP/1.1" 400 -


Bloque:
{'next_id': 'Carrusel | cce128da-fc3c-4ab8-8733-0f93dff9b0ca | 55686726-3982-4003-0323-821800355990 | ZG7udwy8rzhE5xtUnOGl', 'namestate': 'Informativo | 34fa0024-002a-4673-8b70-6419afef7d4d | 55686726-3982-4003-0323-821800355990 | ZG7udwy8rzhE5xtUnOGl', 'contenido': 'Este es el carrusel prueba 222222', 'contenttype': 'text', 'typingtime': '1', 'id_robot': 538, 'blocktype': 'informativo'}
Bloques Bot:
{'Carrusel | 5aaf3158-308e-4b01-9283-f01f32d587c9 | 96628859-0899-7537-4503-489244348545 | ZG7udwy8rzhE5xtUnOGl': {'type': 'slide', 'name': 'Carrusel | 5aaf3158-308e-4b01-9283-f01f32d587c9 | 96628859-0899-7537-4503-489244348545 | ZG7udwy8rzhE5xtUnOGl', 'parameters': {'typingtime': '1', 'nombreVariable': 'variableCarrCarrusel_|_5aaf3158-308e-4b01-9283-f01f32d587c9', 'tipoVariable': 'Variable', 'tipoDato': 'Cadena', 'id_var': 1810, 'id_robot': 538}, 'outputs': {'default_id': 'Carrusel | 5aaf3158-308e-4b01-9283-f01f32d587c9 | 96628859-0899-7537-4503-489244348545 | ZG7udwy8rzhE5xtUnOGl

...

Bloque:
{'next_id': '', 'namestate': 'Salida', 'contenido': '', 'contenttype': 'text', 'typingtime': 3, 'id_robot': 540, 'blocktype': 'informativo'}
Bloques Bot:
{'Carrusel | fdd978d8-d9e9-4f74-8d01-07d1a43ecba8 | 98525253-9080-3718-4541-185145979160 | bSkwpaFVCmMXS8G2PR6k': {'type': 'slide', 'name': 'Carrusel | fdd978d8-d9e9-4f74-8d01-07d1a43ecba8 | 98525253-9080-3718-4541-185145979160 | bSkwpaFVCmMXS8G2PR6k', 'parameters': {'typingtime': '1', 'nombreVariable': 'varCarrusel_|_fdd978d8-d9e9-4f74-8d01-07d1a43ecba83518', 'tipoVariable': 'Variable', 'tipoDato': 'Cadena', 'id_var': 1896, 'id_robot': 540}, 'outputs': {'default_id': 'Carrusel | fdd978d8-d9e9-4f74-8d01-07d1a43ecba8 | 98525253-9080-3718-4541-185145979160 | bSkwpaFVCmMXS8G2PR6k'}}, 'Informativo | 7e74c6f6-46db-4b00-b8b3-07aac064e1de | 98525253-9080-3718-4541-185145979160 | bSkwpaFVCmMXS8G2PR6k': {'type': 'informativo', 'name': 'Informativo | 7e74c6f6-46db-4b00-b8b3-07aac064e1de | 98525253-9080-3718-4541-185145979160 | bSkwpaFVC

127.0.0.1 - - [15/Jun/2021 00:10:37] "POST /insert HTTP/1.1" 200 -


Se creo el chatbot
Datos del ChatBot:
{'name_robot': 'Chatbot TT 2021', 'id_face': '110373671266869', 'block_ini': 'Informativo | a0be25bb-4eb4-49c2-830f-6eb23d64966c | ZG7udwy8rzhE5xtUnOGl', 'type_blocki': 'informativo', 'access_token': 'EAALZCCZBzPwfkBAAtF3N1x3GCvg9UuN7alfdTNGmZC5d96EY7vYBM6hQQedeMtNTZAWxFVFBmRXfT2lp3m970ZCDZBhelEyxvK6PN9gHzEXZBbdxvqXqETQzUffJynrycHPnBBR5GtxJl2ulvP0uXrP0fTyoEmQ5GchMN0ckdunXK6MEd1Q40Cb', 'id_user': 'charlygoldcruz@gmail.com', 'api_nlp': 'http://localhost:8282/'}
{'bloques': {'Carrusel | fdd978d8-d9e9-4f74-8d01-07d1a43ecba8 | 98525253-9080-3718-4541-185145979160 | bSkwpaFVCmMXS8G2PR6k': {'type': 'slide', 'name': 'Carrusel | fdd978d8-d9e9-4f74-8d01-07d1a43ecba8 | 98525253-9080-3718-4541-185145979160 | bSkwpaFVCmMXS8G2PR6k', 'parameters': {'typingtime': '1', 'content': '', 'nombreVariable': 'varCarrusel_|_fdd978d8-d9e9-4f74-8d01-07d1a43ecba89602', 'jsonVariable': {'variable': {'name_var': 'varCarrusel_|_fdd978d8-d9e9-4f74-8d01-07d1a43ecba89602', 'opc_typ

127.0.0.1 - - [15/Jun/2021 00:10:41] "POST /insert HTTP/1.1" 200 -


Bloque:
{'next_id': '', 'namestate': 'Salida', 'contenido': '', 'contenttype': 'text', 'typingtime': 3, 'id_robot': 540, 'blocktype': 'informativo'}
Bloques Bot:
{'Carrusel | fdd978d8-d9e9-4f74-8d01-07d1a43ecba8 | 98525253-9080-3718-4541-185145979160 | bSkwpaFVCmMXS8G2PR6k': {'type': 'slide', 'name': 'Carrusel | fdd978d8-d9e9-4f74-8d01-07d1a43ecba8 | 98525253-9080-3718-4541-185145979160 | bSkwpaFVCmMXS8G2PR6k', 'parameters': {'typingtime': '1', 'nombreVariable': 'varCarrusel_|_fdd978d8-d9e9-4f74-8d01-07d1a43ecba89602', 'tipoVariable': 'Variable', 'tipoDato': 'Cadena', 'id_var': 1910, 'id_robot': 540}, 'outputs': {'default_id': 'Carrusel | fdd978d8-d9e9-4f74-8d01-07d1a43ecba8 | 98525253-9080-3718-4541-185145979160 | bSkwpaFVCmMXS8G2PR6k'}}, 'Informativo | 7e74c6f6-46db-4b00-b8b3-07aac064e1de | 98525253-9080-3718-4541-185145979160 | bSkwpaFVCmMXS8G2PR6k': {'type': 'informativo', 'name': 'Informativo | 7e74c6f6-46db-4b00-b8b3-07aac064e1de | 98525253-9080-3718-4541-185145979160 | bSkwpaFVC